In [2]:
#!/usr/bin/python

import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

data2014 = pd.read_csv("../Week7/Input/2014dataset.csv",low_memory=False)
data2015 = pd.read_csv("../Week7/Input/2015dataset.csv",low_memory=False)
data2016 = pd.read_csv("../Week7/Input/2016dataset.csv",low_memory=False)

mm_cutoff = 2000

important_fields_ind = ["HIOS ID", 
"HIOS INPUTTED INSURANCE COMPANY NAME", 
"STATE", 
"REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)",
"HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
"Not-For-Profit:",
"7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15",
"1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15",
"2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15",]
					
important_fields_sg = ["HIOS ID",
"HIOS INPUTTED INSURANCE COMPANY NAME",
"STATE",
"Not-For-Profit:",
"HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)", 
"7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15", 
"1.1Directpremiumwritten7HealthInsuranceSMALLGROUPTotalasof3/31/15",
"2.16Totalincurredclaims7HealthInsuranceSMALLGROUPTotalasof3/31/15"]

data2014_ind = data2014[important_fields_ind].dropna(how='all')
data2015_ind = data2015[important_fields_ind].dropna(how='all')
data2016_ind = data2016[important_fields_ind].dropna(how='all')

data2014_sg = data2014[important_fields_sg].dropna(how='all')
data2015_sg = data2015[important_fields_sg].dropna(how='all')
data2016_sg = data2016[important_fields_sg].dropna(how='all')

def add_columns_ind(playground1):
    playground1["IndPremiumPMM"] = playground1["1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndCostsPMM"] = playground1["2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndRTPMM"] = playground1["HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndReinsPMM"] = playground1["REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndLossAPMM"] = playground1["IndPremiumPMM"] - playground1["IndCostsPMM"]
    playground1["IndLossBPMM"] = playground1["IndLossAPMM"] + playground1["IndRTPMM"]
    playground1["IndLossCPMM"] = playground1["IndLossBPMM"] + playground1["IndReinsPMM"]

def add_columns_sg(playground1):
    playground1["SGPremiumPMM"] = playground1["1.1Directpremiumwritten7HealthInsuranceSMALLGROUPTotalasof3/31/15"] / playground1["7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15"]
    playground1["SGCostsPMM"] = playground1["2.16Totalincurredclaims7HealthInsuranceSMALLGROUPTotalasof3/31/15"] / playground1["7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15"]
    playground1["SGRTPMM"] = playground1["HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)"] / playground1["7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15"]
    playground1["SGLossAPMM"] = playground1["SGPremiumPMM"] - playground1["SGCostsPMM"]
    playground1["SGLossBPMM"] = playground1["SGLossAPMM"] + playground1["SGRTPMM"]

add_columns_ind(data2014_ind)
add_columns_ind(data2015_ind)
add_columns_ind(data2016_ind)

add_columns_sg(data2014_sg)
add_columns_sg(data2015_sg)
add_columns_sg(data2016_sg)

merged_ind = data2014_ind.merge(data2015_ind, on='HIOS ID', how='outer', suffixes=('_2014', '_2015'))
merged_ind = merged_ind.where(merged_ind["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15_2014"] > 2000).dropna(how='all')
merged_ind = merged_ind.where(merged_ind["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15_2015"] > 2000).dropna(how='all')
merged_ind_final = merged_ind.merge(data2016_ind, on='HIOS ID', how='outer', suffixes=('', '_2016'))
#merged_ind_final = merged_ind_final.where(merged_ind_final["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"] > 2000).dropna(how='all')
merged_ind_final.to_csv("../Week13/IndividualData.csv")

merged_sg = data2014_sg.merge(data2015_sg, on='HIOS ID', how='outer', suffixes=('_2014', '_2015'))
merged_sg = merged_sg.where(merged_sg["7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15_2014"] > 2000).dropna(how='all')
merged_sg = merged_sg.where(merged_sg["7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15_2015"] > 2000).dropna(how='all')
merged_sg_final = merged_sg.merge(data2016_sg, on='HIOS ID', how='outer', suffixes=('', '_2016'))
#merged_sg_final = merged_sg_final.where(merged_sg_final["7.4Membermonths2HealthInsurancesgIVIDUALTotalasof3/31/15"] > 2000).dropna(how='all')
merged_sg_final.to_csv("../Week13/SmallGroupData.csv")

In [23]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ind = pd.read_csv("IndividualData.csv")
sg = pd.read_csv("SmallGroupData.csv")
states = ind["STATE"].drop_duplicates()
ind = ind.merge(sg, on='HIOS ID', how='inner', suffixes=('_x', '_y')).dropna(how='any')

sg = sg.merge(ind, on='HIOS ID', how='inner', suffixes=('_x', '_y')).dropna(how='any')

print("State, Ind Premium, Ind n, SG Premium, SG n")

count = 0
count1 = 0
for index, row in ind.iterrows():
    if row["SGPremiumPMM"] > row["IndPremiumPMM"]:
        count = count + 1
    if row["SGPremiumPMM"] < row["IndPremiumPMM"]:
        count1 = count1 + 1
    
print("Small Group bigger: " + str(count))
print("Individual bigger: " + str(count1))
print("Total: " + str(count + count1))

State, Ind Premium, Ind n, SG Premium, SG n
Small Group bigger: 169
Individual bigger: 51
Total: 220


In [7]:
ind.head()

,Unnamed: 0_x,HIOS ID,HIOS INPUTTED INSURANCE COMPANY NAME_2014_x,STATE_2014_x,REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)_2014,"HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)_2014",Not-For-Profit:_2014_x,7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15_2014,1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15_2014,2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15_2014,...,Not-For-Profit:_y,HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET),7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15,1.1Directpremiumwritten7HealthInsuranceSMALLGROUPTotalasof3/31/15,2.16Totalincurredclaims7HealthInsuranceSMALLGROUPTotalasof3/31/15,SGPremiumPMM,SGCostsPMM,SGRTPMM,SGLossAPMM,SGLossBPMM
0,300.0,47579.0,Chinese Community Health Plan,CA,1824095.22,-16711278.79,No,141014.0,6.026872e+07,2.994971e+07,...,No,-2248025.07,25754.0,9941795.73,7.127228e+06,386.029189,276.742569,-87.288385,109.286620,21.998235
1,368.0,11177.0,MetroPlus Health Plan,NY,4200475.38,-54033594.10,No,399516.0,1.449844e+08,7.712122e+07,...,Yes,-4170812.16,11773.0,5785888.20,1.788156e+06,491.454022,151.886193,-354.269274,339.567829,-14.701444
2,229.0,81941.0,Aetna Health Inc. (a LA corp.),LA,1125984.52,-2764603.70,No,149620.0,2.700270e+07,2.551396e+07,...,No,-644489.03,29833.0,12589367.81,1.021192e+07,421.994697,342.302846,-21.603226,79.691851,58.088626
3,182.0,47342.0,Health Tradition Health Plan,WI,4487321.80,-626288.42,No,37196.0,1.710072e+07,2.052331e+07,...,No,-2172128.98,34451.0,14240989.00,1.239474e+07,413.369394,359.778812,-63.049809,53.590582,-9.459227
4,376.0,23841.0,Aetna Life Insurance Company,FL,32190446.23,20415906.73,No,409905.0,1.527766e+08,1.758660e+08,...,No,2505152.50,60960.0,32168939.93,2.381995e+07,527.705708,390.747174,41.095021,136.958533,178.053555
